# Custom NER for Identifying Diseases and Treatments

This notebook implements a custom Named Entity Recognition (NER) system to identify diseases and treatments from a medical dataset. The dataset is provided in tokenized format, where each word is associated with a label:
- `O` indicates "Other"
- `D` indicates "Disease"
- `T` indicates "Treatment"

## Steps in this Notebook
1. **Data Preprocessing:** Reconstruct sentences and labels from the tokenized dataset.
2. **Concept Identification:** Identify key concepts in the dataset using PoS tagging.
3. **Defining Features for CRF:** Create features for training the CRF model.
4. **Getting Features for Words and Sentences:** Apply feature definitions to all sentences.
5. **Defining Input and Target Variables:** Prepare input features and labels for training and testing.
6. **Building the Model:** Train the CRF model on the training dataset.
7. **Evaluating the Model:** Evaluate the model on the test dataset using F1 score and classification metrics.
8. **Identifying Diseases and Predicted Treatments:** Extract relationships between diseases and treatments using the trained model.


## Step 1: Data Preprocessing
The dataset is provided in tokenized format, where each word is stored on a separate line, and sentences are separated by blank lines. In this step, I will:
1. Reconstruct sentences and labels from the training and testing datasets.
2. Count the number of sentences and labels in the processed datasets.


In [1]:
# Paths to the dataset files
train_sent_path = 'data/train_sent'
train_label_path = 'data/train_label'
test_sent_path = 'data/test_sent'
test_label_path = 'data/test_label'

In [2]:
import re

def filter_no_entity_statements(sentences, labels):
    """
    Filters out sentences and their corresponding labels that do not contain any entity labels ('D' or 'T').

    Parameters:
    sentences (list): A list of tokenized sentences, where each sentence is a list of words.
    labels (list): A list of label sequences, where each sequence corresponds to a sentence.

    Returns:
    tuple: Filtered lists of sentences and labels.
    """
    filtered_sentences = []
    filtered_labels = []

    for sentence, label_sequence in zip(sentences, labels):
        # Check if the label sequence contains any 'D' or 'T' labels
        if any(label in {'D', 'T'} for label in label_sequence):
            filtered_sentences.append(sentence)
            filtered_labels.append(label_sequence)

    return filtered_sentences, filtered_labels

def clean_sentence(sentence, labels):
    """
    Cleans a tokenized sentence by normalizing, removing noise, and correcting token splits.
    Adjusts the labels accordingly to ensure alignment with cleaned tokens.

    Parameters:
    sentence (list): A list of tokens (words) in the sentence.
    labels (list): A list of labels corresponding to the tokens.

    Returns:
    tuple: A cleaned list of tokens and their adjusted labels.
    """
    cleaned_sentence = []
    cleaned_labels = []

    for token, label in zip(sentence, labels):
        # Remove special characters except hyphens in compound words
        cleaned_token = re.sub(r'[^\w\-]', '', token)

        # Preserve meaningful hyphenated words, otherwise split on hyphen
        if '-' in cleaned_token and len(cleaned_token.split('-')) > 1:
            sub_tokens = cleaned_token.split('-')
            if all(len(sub) > 1 for sub in sub_tokens):  # If all parts are meaningful
                cleaned_sentence.append(cleaned_token)
                cleaned_labels.append(label)
            else:
                for sub_token in sub_tokens:
                    cleaned_sentence.append(sub_token)
                    cleaned_labels.append(label)
        else:
            # Lowercase all tokens except acronyms or proper nouns
            if cleaned_token.isupper() and len(cleaned_token) > 1:  # Acronyms like "HIV"
                cleaned_sentence.append(cleaned_token)
                cleaned_labels.append(label)
            else:
                cleaned_sentence.append(cleaned_token.lower())
                cleaned_labels.append(label)

    # Filter out empty tokens or noise tokens
    final_sentence = []
    final_labels = []
    for tok, lab in zip(cleaned_sentence, cleaned_labels):
        if tok and re.search(r'[a-zA-Z]', tok):  # Ensure valid tokens remain
            final_sentence.append(tok)
            final_labels.append(lab)

    return final_sentence, final_labels


def clean_dataset(sentences, labels):
    """
    Cleans a dataset of tokenized sentences and adjusts their corresponding labels.

    Parameters:
    sentences (list): A list of sentences, where each sentence is a list of tokens.
    labels (list): A list of label sequences, where each sequence corresponds to a sentence.

    Returns:
    tuple: A cleaned list of sentences and their adjusted labels.
    """
    cleaned_sentences = []
    cleaned_labels = []

    for sentence, label_sequence in zip(sentences, labels):
        cleaned_sentence, cleaned_label = clean_sentence(sentence, label_sequence)
        cleaned_sentences.append(cleaned_sentence)
        cleaned_labels.append(cleaned_label)

    return cleaned_sentences, cleaned_labels

def process_data(file_path):
    """
    Read a dataset file and reconstruct sentences or labels.

    Parameters:
    file_path (str): Path to the file containing data in tokenized format.

    Returns:
    list: A list of sentences or labels reconstructed from the file.
    """
    sentences = []
    current_sentence = []

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line == "":  # A blank line indicates the end of a sentence
                if current_sentence:
                    sentences.append(current_sentence)
                    current_sentence = []
            else:
                current_sentence.append(line)
        if current_sentence:  # Add the last sentence if the file does not end with a blank line
            sentences.append(current_sentence)

    return sentences

In [3]:
# Process train and test datasets
train_sentences = process_data(train_sent_path)
train_labels = process_data(train_label_path)
test_sentences = process_data(test_sent_path)
test_labels = process_data(test_label_path)

# Verify by printing counts
print(f"Number of sentences in train dataset: {len(train_sentences)}")
print(f"Number of label lines in train dataset: {len(train_labels)}")
print(f"Number of sentences in test dataset: {len(test_sentences)}")
print(f"Number of label lines in test dataset: {len(test_labels)}")

Number of sentences in train dataset: 2599
Number of label lines in train dataset: 2599
Number of sentences in test dataset: 1056
Number of label lines in test dataset: 1056


### Clean data upfront for noise

In [39]:
# train_sentences, train_labels = clean_dataset(train_sentences, train_labels)
# test_sentences, test_labels = clean_dataset(test_sentences, test_labels)

# train_sentences, train_labels = filter_no_entity_statements(sentences=train_sentences, labels=train_labels)
# test_sentences, test_labels = filter_no_entity_statements(sentences=test_sentences, labels=test_labels)

# Verify by printing counts
print(f"Number of sentences in train dataset: {len(train_sentences)}")
print(f"Number of label lines in train dataset: {len(train_labels)}")

print(f"Number of sentences in test dataset: {len(test_sentences)}")
print(f"Number of label lines in test dataset: {len(test_labels)}")

Number of sentences in train dataset: 2599
Number of label lines in train dataset: 2599
Number of sentences in test dataset: 1056
Number of label lines in test dataset: 1056


NOTE : I concluded on not cleaning any data upfront as all attempts were detremental to model performance.

## Step 2: Concept Identification
In this step, I will identify key concepts (e.g., diseases and treatments) from the dataset by:
1. Performing Part-of-Speech (PoS) tagging on the text data.
2. Extracting tokens with PoS tags corresponding to nouns (`NOUN` and `PROPN`).
3. Counting the frequency of these tokens across the entire dataset (both training and testing data).
4. Printing the top 25 most frequently mentioned concepts.


In [5]:
# For formatting outputs
from tabulate import tabulate

In [6]:
import spacy
from spacy.cli import download
try:
    spacy.load("en_core_web_sm")
except:
    download("en_core_web_sm")
    exit(1)

In [7]:
import spacy
from collections import Counter

# Load spaCy model for PoS tagging
nlp = spacy.load("en_core_web_sm")

def extract_noun_phrases(sentences):
    """
    Extract nouns and proper nouns from the given sentences.

    Parameters:
    sentences (list): A list of tokenized sentences.

    Returns:
    list: A list of nouns and proper nouns extracted from the sentences.
    """
    nouns = []
    for sentence in sentences:
        doc = nlp(" ".join(sentence))
        for token in doc:
            if token.pos_ in ["NOUN", "PROPN"]:  # Select nouns and proper nouns
                nouns.append(token.text.lower())
    return nouns

# Combine training and testing sentences for concept identification
all_sentences = train_sentences + test_sentences

# Extract nouns and calculate their frequencies
nouns = extract_noun_phrases(all_sentences)
noun_frequencies = Counter(nouns)

In [8]:
# Print the top 25 most common nouns
table_data = [[i, concept, freq] for i, (concept, freq) in enumerate(noun_frequencies.most_common(25), start=1)]
print(tabulate(table_data, headers=['#', "Concept", "Frequency"], tablefmt="github"))

|   # | Concept      |   Frequency |
|-----|--------------|-------------|
|   1 | patients     |         507 |
|   2 | treatment    |         304 |
|   3 | %            |         247 |
|   4 | cancer       |         211 |
|   5 | therapy      |         177 |
|   6 | study        |         174 |
|   7 | disease      |         149 |
|   8 | cell         |         142 |
|   9 | lung         |         118 |
|  10 | results      |         116 |
|  11 | group        |         111 |
|  12 | effects      |          99 |
|  13 | gene         |          91 |
|  14 | chemotherapy |          91 |
|  15 | use          |          87 |
|  16 | effect       |          82 |
|  17 | women        |          81 |
|  18 | analysis     |          76 |
|  19 | risk         |          74 |
|  20 | surgery      |          73 |
|  21 | cases        |          72 |
|  22 | p            |          72 |
|  23 | rate         |          68 |
|  24 | survival     |          67 |
|  25 | response     |          66 |


In [9]:
# Print the top 25 least common nouns
table_data = [[i, concept, freq] for i, (concept, freq) in enumerate(noun_frequencies.most_common()[:-26:-1], start=1)]
print(tabulate(table_data, headers=['#', "Concept", "Frequency"], tablefmt="github"))

|   # | Concept           |   Frequency |
|-----|-------------------|-------------|
|   1 | abortion          |           1 |
|   2 | myeloma           |           1 |
|   3 | tandem            |           1 |
|   4 | occlusions        |           1 |
|   5 | thrombogenicity   |           1 |
|   6 | vasoreactivity    |           1 |
|   7 | epoetin           |           1 |
|   8 | timolol           |           1 |
|   9 | tartrate          |           1 |
|  10 | brimonidine       |           1 |
|  11 | poliovirus        |           1 |
|  12 | poliomyelitis     |           1 |
|  13 | celecoxib         |           1 |
|  14 | formoterol        |           1 |
|  15 | dry               |           1 |
|  16 | levodopa          |           1 |
|  17 | methyltransferase |           1 |
|  18 | catechol          |           1 |
|  19 | tolcapone         |           1 |
|  20 | colonoscopy       |           1 |
|  21 | malathion         |           1 |
|  22 | spoon             |       

## Step 3: Defining Features for CRF
This step involves defining the features for training the Conditional Random Field (CRF) model. The features will capture:
1. Word-level attributes (e.g., lowercase form, capitalization, title-case, digits).
2. Part-of-Speech (PoS) tags for the current word, as well as preceding and succeeding words.
3. Contextual information, such as bigrams and sentence boundaries (start and end indicators).
The features are essential for capturing the relationships and contexts necessary for NER.


### Explanation of Feature Selection

1. **Word-Level Features**: Capture basic properties of words, such as case, prefixes, suffixes, and the presence of hyphens, to recognize patterns in medical terms.
2. **POS and Dependency Features**: Utilize spaCy's Part-of-Speech tagging and dependency parsing to capture syntactic roles and relationships in the sentence.
3. **Contextual Features**: Include the previous and next words and their grammatical roles to provide a broader context for each word.
4. **N-grams**: Create bigram features to identify relationships between consecutive words, such as compound terms or descriptors.
5. **Phrase Boundary Features**: Detect modifiers (e.g., adjectives or compounds) that indicate the start or end of an entity phrase.
6. **Sentence Position**: Use markers for the beginning and end of sentences to help identify potential boundaries of entities.

In [10]:
def word2features(sentence, i):
    """
    Generate features for a single word in a sentence with context relationships.

    Parameters:
    sentence (list): A list of tokens (words) in the sentence.
    i (int): Index of the word in the sentence.

    Returns:
    dict: A dictionary of features for the word.
    """
    word = sentence[i]
    features = {
        'word.lower()': word.lower(),  # Lowercased word
        'word.prefix': word[:3].lower(),  # First 3 characters
        'word.suffix': word[-3:].lower(),  # Last 3 characters
    }

    # PoS tagging using spaCy
    doc = nlp(" ".join(sentence))
    token = doc[i]
    features['pos'] = token.pos_
    features['dep'] = token.dep_
    features['head'] = token.head.text.lower()  # Head word
    features['head.pos'] = token.head.pos_

    # Features for the beginning and end of a sentence
    features['BOS'] = (i == 0)  # Beginning of the sentence
    features['EOS'] = (i == len(sentence) - 1)  # End of the sentence

    # Features for previous word
    if i > 0:
        prev_token = doc[i - 1]
        features.update({
            'prev_word.lower()': sentence[i - 1].lower(),
            'prev_word.pos': prev_token.pos_,
            'prev_word.dep': prev_token.dep_,
        })
    else:
        features['prev_word.lower()'] = '<START>'

    # Features for next word
    if i < len(sentence) - 1:
        next_token = doc[i + 1]
        features.update({
            'next_word.lower()': sentence[i + 1].lower(),
            'next_word.pos': next_token.pos_,
            'next_word.dep': next_token.dep_,
        })
    else:
        features['next_word.lower()'] = '<END>'

    # N-gram features
    if i > 0:
        features['bigram.prev'] = sentence[i - 1].lower() + '_' + word.lower()
    if i < len(sentence) - 1:
        features['bigram.next'] = word.lower() + '_' + sentence[i + 1].lower()
    if i > 1:
        features['trigram.prev'] = sentence[i - 2].lower() + '_' + sentence[i - 1].lower() + '_' + word.lower()
    if i < len(sentence) - 2:
        features['trigram.next'] = word.lower() + '_' + sentence[i + 1].lower() + '_' + sentence[i + 2].lower()

    # Phrase boundary features
    features['is_descriptor'] = token.dep_ in ['amod', 'compound']

    return features

def sent2features(sentence):
    """
    Generate features for all words in a sentence.

    Parameters:
    sentence (list): A list of tokens (words) in the sentence.

    Returns:
    list: A list of dictionaries, each containing features for a word.
    """
    return [word2features(sentence, i) for i in range(len(sentence))]

## Step 4: Getting Features for Words and Sentences
Using the feature extraction functions defined earlier, I will generate features for all sentences in the training and testing datasets. This involves:
1. Applying `sent2features` to each sentence.
2. Preparing the data in a format suitable for training and evaluating the CRF model.


In [11]:
# TODO - run feature extraction in parallel to speed up
# from joblib import Parallel, delayed

# def prepare_features_and_labels(sentences, labels):
#     """
#     Generate features and labels for all sentences in the dataset.

#     Parameters:
#     sentences (list): A list of sentences, where each sentence is a list of tokens (words).
#     labels (list): A list of label sequences, where each sequence corresponds to a sentence.

#     Returns:
#     tuple: A tuple containing:
#         - features (list): A list of feature dictionaries for each sentence.
#         - labels (list): A list of label sequences for each sentence.
#     """
#     # Parallelize the sentence feature extraction using threads
#     features = Parallel(n_jobs=-1, prefer="threads")(delayed(sent2features)(sentence) for sentence in sentences)
#     return features, labels

# # Prepare features and labels for the train dataset
# train_features, train_labels = prepare_features_and_labels(train_sentences, train_labels)

# # Prepare features and labels for the test dataset
# test_features, test_labels = prepare_features_and_labels(test_sentences, test_labels)

In [12]:
def prepare_features_and_labels(sentences, labels):
    """
    Generate features and labels for all sentences in the dataset.

    Parameters:
    sentences (list): A list of sentences, where each sentence is a list of tokens (words).
    labels (list): A list of label sequences, where each sequence corresponds to a sentence.

    Returns:
    tuple: A tuple containing:
        - features (list): A list of feature dictionaries for each sentence.
        - labels (list): A list of label sequences for each sentence.
    """
    features = [sent2features(sentence) for sentence in sentences]
    return features, labels

# Prepare features and labels for the train dataset
train_features, train_labels = prepare_features_and_labels(train_sentences, train_labels)

# Prepare features and labels for the test dataset
test_features, test_labels = prepare_features_and_labels(test_sentences, test_labels)

## Step 5: Defining Input and Target Variables
In this step, I will define the input features and target labels for the CRF model:
1. Input Variables: Features extracted for each word in the sentences.
2. Target Variables: Corresponding labels (`O`, `D`, `T`) for each word in the sentences.

Additionally, I will display a random example from the training dataset in a tabular format to inspect the features and labels.


In [13]:
import random
from tabulate import tabulate

# Display the number of samples for training and testing
print(f"Number of training samples: {len(train_features)}")
print(f"Number of testing samples: {len(test_features)}")

Number of training samples: 2599
Number of testing samples: 1056


In [49]:
# Function to display features and labels in a tabular format
def display_random_example(features, labels, sentences):
    """
    Display a random example from the dataset in a tabular format.

    Parameters:
    features (list): List of feature dictionaries for the dataset.
    labels (list): List of label sequences corresponding to the features.
    sentences (list): List of tokenized sentences.
    """
    # Select a random example
    random_index = random.randint(0, len(features) - 1)
    example_features = features[random_index]
    example_labels = labels[random_index]
    example_sentence = sentences[random_index]
    
    # Prepare the data for tabulation
    table_data = []
    for i, (word, label, feature) in enumerate(zip(example_sentence, example_labels, example_features)):
        row = [i + 1, word, label] + [f"{key}: {value}" for key, value in feature.items()]
        table_data.append(row)
    
    # Define headers for the table
    headers = ["Index", "Word", "Label"] + [f"Feature {i + 1}" for i in range(len(example_features[0]))]

    # Display the table using tabulate
    print(f"\nRandom Example from Training Set (Index {random_index}):")
    print(tabulate(table_data, headers=headers, tablefmt="github"))

# Display a random example from the training set
display_random_example(train_features, train_labels, train_sentences)


Random Example from Training Set (Index 2412):
|   Index | Word         | Label   | Feature 1                  | Feature 2        | Feature 3        | Feature 4   | Feature 5     | Feature 6         | Feature 7      | Feature 8   | Feature 9   | Feature 10                      | Feature 11                      | Feature 12              | Feature 13                      | Feature 14                       | Feature 15                                     | Feature 16                             |
|---------|--------------|---------|----------------------------|------------------|------------------|-------------|---------------|-------------------|----------------|-------------|-------------|---------------------------------|---------------------------------|-------------------------|---------------------------------|----------------------------------|------------------------------------------------|----------------------------------------|
|       1 | Direct       | T       | word.lower(

## Step 6: Building the Model

In this step, I perform model selection by evaluating a Conditional Random Field (CRF) model with various combinations of hyperparameters. These include optimization algorithms (`lbfgs`, `arow`, `pa`), regularization coefficients (`c1`, `c2`), and maximum iterations. 

The goal is to identify the best-performing model based on the F1-score for the `D` (Disease) label. The best model is then assigned to the variable `crf_model` for further use.


I am not evaluating `lbfgs` as I already tested it to conclude it is not a candidate.

In [15]:
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_classification_report

# Function to train and evaluate a CRF model with specified parameters
def train_crf(algorithm, c1, c2, max_iterations, train_features, train_labels, test_features, test_labels):
    """
    Train and evaluate a CRF model.

    Parameters:
    - algorithm (str): The optimization algorithm (e.g., 'lbfgs', 'arow', 'pa').
    - c1 (float): Coefficient for L1 regularization (only for lbfgs).
    - c2 (float): Coefficient for L2 regularization (only for lbfgs).
    - max_iterations (int): Maximum number of iterations for training.
    - train_features (list): Features for the training data.
    - train_labels (list): Labels for the training data.
    - test_features (list): Features for the test data.
    - test_labels (list): Labels for the test data.

    Returns:
    - f1_score_d (float): F1-score for the 'D' (Disease) label on the test set.
    - model: The trained CRF model.
    - report: Classification report.
    """
    if algorithm in ["arow", "pa"]:
        crf = CRF(
            algorithm=algorithm,
            max_iterations=max_iterations,
            all_possible_transitions=True
        )
    else:
        crf = CRF(
            algorithm=algorithm,
            c1=c1,
            c2=c2,
            max_iterations=max_iterations,
            all_possible_transitions=True
        )

    crf.fit(train_features, train_labels)
    predictions = crf.predict(test_features)
    report = flat_classification_report(test_labels, predictions, output_dict=True)
    f1_score_d = report['D']['f1-score']
    return f1_score_d, crf, report

# Define parameters for grid search
algorithms = ['arow', 'pa']
max_iterations_values = [100, 125, 150, 175, 200, 250, 300]

parameter_combinations = (
    [(alg, None, None, max_iter) for alg in ['arow', 'pa'] for max_iter in max_iterations_values]
)

# Grid search for best model
best_f1_d = 0
best_model = None
best_params = None
best_report = None

for algorithm, c1, c2, max_iterations in parameter_combinations:
    if algorithm in ["arow", "pa"]:
        print(f"Evaluating: Algorithm={algorithm}, max_iterations={max_iterations}")
    # else:
    #     print(f"Evaluating: Algorithm={algorithm}, c1={c1}, c2={c2}, max_iterations={max_iterations}")
    try:
        f1_d, model, report = train_crf(
            algorithm, c1, c2, max_iterations,
            train_features, train_labels, test_features, test_labels
        )
        if f1_d > best_f1_d:
            best_f1_d = f1_d
            best_model = model
            best_params = (algorithm, c1, c2, max_iterations)
            best_report = report
    except Exception as e:
        print(f"Error with combination Algorithm={algorithm}, c1={c1}, c2={c2}, max_iterations={max_iterations}: {e}")


Evaluating: Algorithm=arow, max_iterations=100
Evaluating: Algorithm=arow, max_iterations=125
Evaluating: Algorithm=arow, max_iterations=150
Evaluating: Algorithm=arow, max_iterations=175
Evaluating: Algorithm=arow, max_iterations=200
Evaluating: Algorithm=arow, max_iterations=250
Evaluating: Algorithm=arow, max_iterations=300
Evaluating: Algorithm=pa, max_iterations=100
Evaluating: Algorithm=pa, max_iterations=125
Evaluating: Algorithm=pa, max_iterations=150
Evaluating: Algorithm=pa, max_iterations=175
Evaluating: Algorithm=pa, max_iterations=200
Evaluating: Algorithm=pa, max_iterations=250
Evaluating: Algorithm=pa, max_iterations=300


In [16]:
best_model

CRF(algorithm='pa', all_possible_transitions=True, max_iterations=250)

In [24]:
# Hard coded to avoid confusion 
crf_model = CRF(
            algorithm='pa',
            max_iterations=250,
            all_possible_transitions=True
)

crf_model.fit(train_features, train_labels)
crf_model

CRF(algorithm='pa', all_possible_transitions=True, max_iterations=250)

## Step 7: Evaluating the Model
In this step, I will evaluate the CRF model's performance using the test dataset. The model will:
1. Predict labels for each token in the test sentences.
2. Calculate the F1 score for overall performance.
3. Display a detailed classification report to analyze the model's predictions for each label (`O`, `D`, `T`).


In [32]:
from sklearn_crfsuite import metrics

# Predict labels for the test dataset
test_predictions = crf_model.predict(test_features)

# Evaluate the model using the F1 score
# f1_score = metrics.flat_f1_score(
#     test_labels, test_predictions, average='weighted', labels=crf_model.classes_
# )

# print(f"F1 Score: {f1_score:.3f}")

# Print classification report for detailed evaluation
classification_report = metrics.flat_classification_report(
    test_labels, test_predictions, labels=crf_model.classes_, digits=2
)
print("Classification Report:")
print(classification_report)

Classification Report:
              precision    recall  f1-score   support

           O       0.94      0.98      0.96     16127
           D       0.80      0.65      0.72      1450
           T       0.80      0.56      0.66      1041

    accuracy                           0.93     18618
   macro avg       0.85      0.73      0.78     18618
weighted avg       0.92      0.93      0.92     18618



### **Model Performance Summary**

#### **Overall Performance**
- **Accuracy**: **0.93**
- **Macro Average F1-Score**: **0.78**
- **Weighted Average F1-Score**: **0.92**

The model demonstrates strong performance in identifying entities (`D`, `T`) and non-entities (`O`). While the majority class (`O`) maintains high precision, recall, and F1-scores, the minority classes (`D` and `T`) exhibit moderate performance with room for improvement in recall.

---

#### **Class-Wise Performance**
1. **`O` (Other) Class**:
   - **Precision**: **0.94**
   - **Recall**: **0.98**
   - **F1-Score**: **0.96**
   - The model performs exceptionally well for the majority class, with very few false positives or false negatives.

2. **`D` (Disease) Class**:
   - **Precision**: **0.80**
   - **Recall**: **0.65**
   - **F1-Score**: **0.72**
   - Precision is reasonably high, indicating fewer false positives. However, recall is lower, suggesting the model misses some disease entities.

3. **`T` (Treatment) Class**:
   - **Precision**: **0.80**
   - **Recall**: **0.56**
   - **F1-Score**: **0.66**
   - Similar to diseases, the treatment class has good precision but struggles with recall, indicating challenges in capturing all relevant treatments.

---

#### **Key Observations**
1. **Strengths**:
   - High precision and recall for the `O` class reflect robust handling of non-entity tokens.
   - Good precision for `D` and `T` classes indicates the model minimizes false positives for diseases and treatments.

2. **Weaknesses**:
   - Recall for `D` and `T` remains a challenge, with the model missing several relevant entities.
   - The moderate macro F1-score (**0.78**) reflects the variability in performance across classes.

---

#### **Conclusion**
The model's performance on non-entity tokens is excellent, and it maintains good precision for entity detection. However, improving recall for the `D` and `T` classes is essential to enhance its ability to identify more entities accurately. This suggests a need for further refinements, such as enhancing context-aware features or adjusting the training dataset.

## Step 8: Identifying Diseases and Predicted Treatments
In this step, I will extract diseases and their corresponding treatments from the test dataset using the trained CRF model. The output will be structured as a dictionary, where:
- Each disease (label `D`) is a key.
- Treatments (label `T`) associated with the disease are the values.
Additionally, the results for the specific disease "hereditary retinoblastoma" will be explicitly extracted to meet the assignment's requirements.


In [27]:
from collections import defaultdict
import spacy
import re

# Load spaCy's small English model for dependency parsing
nlp = spacy.load("en_core_web_sm")

def extract_diseases_and_treatments(sentences, predictions):
    """
    Extract diseases and treatments, including descriptive multi-word entities,
    with reduced noise using dependency parsing and validation.

    Parameters:
    sentences (list): A list of tokenized sentences.
    predictions (list): A list of predicted label sequences for each sentence.

    Returns:
    dict: A dictionary where keys are diseases (D) with descriptors and values are lists of treatments (T).
    """
    disease_treatment_map = defaultdict(list)

    def is_valid_entity(entity):
        """
        Validate if the extracted entity is meaningful.

        Parameters:
        entity (str): The entity to validate.

        Returns:
        bool: True if the entity is valid, False otherwise.
        """
        # Disallow entities with invalid characters or overly short entities
        if re.search(r"[()\d]", entity) or len(entity.split()) < 1 or re.match(r"^[A-Z]\.$", entity):
            return False
        # Exclude overly generic terms
        if entity.lower() in ["disease", "cancer", "advanced disease"]:
            return False
        return True

    def is_valid_treatment(treatment):
        """
        Validate if the extracted treatment is meaningful.

        Parameters:
        treatment (str): The treatment to validate.

        Returns:
        bool: True if the treatment is valid, False otherwise.
        """
        # Exclude generic terms and overly short treatments
        invalid_terms = {"and", "with", "the", "of"}
        return treatment.isalpha() and len(treatment) > 2 and treatment.lower() not in invalid_terms

    for sentence, prediction in zip(sentences, predictions):
        # Convert the tokenized sentence into a spaCy Doc object for dependency parsing
        doc = nlp(" ".join(sentence))

        current_disease = None
        for idx, (word, label) in enumerate(zip(sentence, prediction)):
            if label == "D":  # Identify disease
                # Start forming a multi-word entity
                token = doc[idx]
                descriptor = set()

                # Add adjectives or compound descriptors linked to the disease
                for child in token.children:
                    if child.dep_ in ["amod", "compound"] and child.pos_ in ["ADJ", "NOUN"]:
                        descriptor.add(child.text)

                # Check for preceding descriptors in the sentence
                j = idx - 1
                while j >= 0 and prediction[j] == "O":
                    prev_token = doc[j]
                    if prev_token.dep_ in ["amod", "compound"] and prev_token.pos_ in ["ADJ", "NOUN"]:
                        descriptor.add(sentence[j])
                    j -= 1

                # Combine descriptor with the disease
                descriptor_list = list(descriptor)
                current_disease = " ".join(descriptor_list + [word])

                # Include subsequent words labeled as `D` to form a multi-word entity
                k = idx + 1
                while k < len(sentence) and prediction[k] == "D":
                    current_disease += f" {sentence[k]}"
                    k += 1

                # Skip to the last word of the entity
                idx = k - 1

                # Validate disease entity
                if not is_valid_entity(current_disease):
                    current_disease = None

            elif label == "T" and current_disease:  # Associate treatment with the disease
                if is_valid_treatment(word):
                    disease_treatment_map[current_disease].append(word)

    # Post-process the map to remove non-alphabetic treatments and normalize phrases
    final_map = {}
    for disease, treatments in disease_treatment_map.items():
        meaningful_treatments = list(set(t for t in treatments if is_valid_treatment(t)))  # Deduplicate treatments
        if is_valid_entity(disease):
            final_map[disease] = meaningful_treatments

    return final_map

In [ ]:
# Extract diseases and treatments using test sentences and predictions
# disease_treatment_dict = extract_diseases_and_treatments(train_sentences, train_labels)
disease_treatment_dict = extract_diseases_and_treatments(test_sentences, test_predictions)

In [51]:
print("\nComplete Disease-Treatment Dictionary:")
table_data = [[i + 1, disease, ', '.join(treatments)] for i, (disease, treatments) in enumerate(disease_treatment_dict.items())]
print(tabulate(table_data, headers=["#", "Disease", "Treatments"], tablefmt="github"))


Complete Disease-Treatment Dictionary:
|   # | Disease                                   | Treatments                                                                                          |
|-----|-------------------------------------------|-----------------------------------------------------------------------------------------------------|
|   1 | diabetes gestational cases                | glycemic, control, good                                                                             |
|   2 | hereditary retinoblastoma                 | radiotherapy                                                                                        |
|   3 | myocardial infarction                     | aspirin                                                                                             |
|   4 | hemorrhagic stroke                        | accelerated, alteplase, infusion                                                                    |
|   5 | intracranial hemorrhage     

In [30]:
# import pandas as pd
# disease_treatment_data = pd.DataFrame(
#     data = table_data,
#     columns=["#", "Disease", "Treatments"]
# )

# disease_treatment_data.set_index('#')    

In [52]:
# Display results for "hereditary retinoblastoma"
specific_disease = "hereditary retinoblastoma"
specific_treatments = disease_treatment_dict.get(specific_disease, [])

if specific_treatments:
    print(f"Predicted treatments for the disease '{specific_disease}': {', '.join(specific_treatments)}")
else:
    print(f"No treatments found for the disease '{specific_disease}'.")

Predicted treatments for the disease 'hereditary retinoblastoma': radiotherapy
